In [149]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 데이터 불러오기

In [150]:
import pandas as pd
DATA_PATH = '/content/drive/MyDrive/DeliverEase/'
hub = pd.read_excel(DATA_PATH+'CJ_Hub.xlsx')

In [151]:
hub.head()

,시/도,터미널종류,터미널이름,주소,위도,경도,담당지
0,경기도,메인허브터미널,MegaHub 곤지암,경기도 광주시 초월읍 산수로 642-70(신월리 794),NaN,NaN,수도권
1,경기도,메인허브터미널,군포Hub,경기도 군포시 부곡동 451번지 차동,NaN,NaN,수도권
2,대전광역시,메인허브터미널,대전Hub,대전 대덕구 대덕대로1447번길 39,NaN,NaN,전국(?)
3,충청북도,메인허브터미널,옥천Hub,충청북도 옥천군 이원면 건진2길 65,NaN,NaN,"서울 서남부, 전국"
4,경기도,메인허브터미널,용인Hub,경기도 용인시 처인구 원삼면 미평리 75번지,NaN,NaN,수도권


## 서울시 서브 터미널 담당구별 데이터 전처리

### 컬럼 추출
-hub 데이터는 터미널 이름과 담당지, 담당구만 사용할 것이기 때문에 필요한 컬럼만 추출한다.
- 컬럼 추출 후 당장 사용할 서울시 sub 데이터를 추출

In [152]:
hub = hub[['터미널이름','담당지']]

In [153]:
seoul_hub=hub[hub['담당지']=='서울특별시']

In [154]:
seoul_hub.head()

,터미널이름,담당지
18,강남ASub,서울특별시
19,강남BSub,서울특별시
20,강동A Sub,서울특별시
21,송파C Sub,서울특별시
22,강북Sub,서울특별시


### 담당구 컬럼 생성
- 현재의 sub 데이터는 담당지가 서울특별시로 정리되어 있을 뿐 지역구별이 아니기 때문에 담당구 컬럼을 만들고 서브 이름의 앞글자 두개를 가져와 담당구를 생성하는 코드를 작성한다.

In [155]:
seoul_hub['담당구']=seoul_hub['터미널이름'].str[0:2]+"구"

<ipython-input-155-9fe1c2224060>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seoul_hub['담당구']=seoul_hub['터미널이름'].str[0:2]+"구"


In [156]:
seoul_hub.head()

,터미널이름,담당지,담당구
18,강남ASub,서울특별시,강남구
19,강남BSub,서울특별시,강남구
20,강동A Sub,서울특별시,강동구
21,송파C Sub,서울특별시,송파구
22,강북Sub,서울특별시,강북구


### 데이터 수정
- 동대문, 서대문 등의 세글자 지역구는 데이터를 제거하고 직접 데이터를 작성해 추가한다.

In [157]:
seoul_hub=seoul_hub[seoul_hub['담당구'] != '영등구']
seoul_hub=seoul_hub[seoul_hub['담당구'] != '서대구']
seoul_hub=seoul_hub[seoul_hub['담당구'] != '동대구']

In [158]:
seoul_hub.head()

,터미널이름,담당지,담당구
18,강남ASub,서울특별시,강남구
19,강남BSub,서울특별시,강남구
20,강동A Sub,서울특별시,강동구
21,송파C Sub,서울특별시,송파구
22,강북Sub,서울특별시,강북구


In [159]:
add_data={
    '터미널이름':['동대문Sub','서대문Sub','영등포Sub','금천Sub'],
    '담당지':['서울특별시','서울특별시','서울특별시','서울특별시'],
    '담당구':['동대문구', '서대문구', '영등포구', '금천구']
}
add_data = pd.DataFrame(add_data)


In [160]:
seoul_hub = pd.concat([seoul_hub, add_data])

In [161]:
seoul_hub.head()

,터미널이름,담당지,담당구
18,강남ASub,서울특별시,강남구
19,강남BSub,서울특별시,강남구
20,강동A Sub,서울특별시,강동구
21,송파C Sub,서울특별시,송파구
22,강북Sub,서울특별시,강북구


### 중복값 처리
- sub의 위치가 아닌 담당구로 데이터를 정리하기 때문에 중복되는 담당구를 가진 터미널들을 groupby를 통해 하나로 정리해준다.

In [162]:
seoul_hub['터미널이름'] = seoul_hub.groupby('담당구')['터미널이름'].transform(lambda x: ' & '.join(x))
# 중복값 제거
seoul_hub.drop_duplicates(subset='담당구', keep='first', inplace=True)
seoul_hub.reset_index().drop(columns='index', axis=1, inplace=True)

In [163]:
seoul_sub = seoul_hub.drop(columns='담당지', axis=1)

In [164]:
seoul_sub

,터미널이름,담당구
18,강남ASub & 강남BSub,강남구
20,강동A Sub,강동구
21,송파C Sub,송파구
22,강북Sub,강북구
23,강서ASub & 강서BSub,강서구
25,관악1Sub,관악구
26,광진BSub & 광진ASub,광진구
28,구로Sub,구로구
29,노원Sub,노원구
30,도봉Sub,도봉구


In [166]:
from google.colab import files
seoul_sub.to_csv('seoul_sub.csv', index=False)
#files.download('seoul_hub.csv')

### 인코딩 문제 해결
- to_csv를 통해 만든 csv 파일을 다운로드 받았는데 excel에서 확인하니 내용이 깨져있었다.
- 확장자 변경을 통해서 해결
  - 로컬 다운로드 파일의 확장자를 .csv 에서 .txt로 바꾼다.
  - 메모장에서 다른 이름으로 저장을 클릭하고 인코딩 방식을 UTF-8이나 ANSI로 변경한다.
  - 저장 후 확장자를 .csv로 다시 바꿔준다.
  - 위의 과정을 했음에도 실패시에는 다운로드 파일을 복사후 다른 폴더에서 똑같은 작업을 수행한다.
